In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import cftime


## Define folders

In [ ]:
dir_SLAND    = '/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_forest   = '/Data/forest_masks/'
dir_area     = '/Data/gridarea/'
dir_grids    = '/Data/grids/'
dir_tmp      = '/Data/tmp/'


## Regrid and collect weighted SLAND

In [ ]:
time_sta = '2001'
time_end = '2015'

#Define models
models = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
          'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs'] #'CLASSIC-N', 'ORCHIDEE', 

experiment = 'S2'

#Select grid size
gridsize = '360x720'
gridsize = '360x720-ForestMask'

#Define compression for output NetCDF file
comp = dict(zlib=True, complevel=3)

#File name of grid file
file_grid_common = dir_grids + 'grid_xy_' + gridsize

#Loop over models
create = 1
for model in models:
    
    print(model)
    
    #Read SLAND data
    fname = dir_SLAND + model + '_' + experiment + '_SLAND_NBP.nc'
    data_SLAND = xr.open_dataset(fname)
    
    #Read area
    fname_area = dir_area + 'gridarea_' + model + '.nc'
    data_area = xr.open_dataset(fname_area)
    
    #Select and average over time
    data_SLAND = data_SLAND.sel(time=slice(time_sta, time_end))    
    data_SLAND = data_SLAND.mean('time')
    
    #Define file name for forest weights
    fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_grid_' + model + '.nc'
    
    #File name of grid file
    file_grid = dir_grids + 'grid_xy_' + model
    
    #Distinguish whethera weighing mask exists or not for each model
    if os.path.exists(fname_weight):

        #Flag for deleting files
        del_files = 1

        #Define file names for remapping (in case different Trendy versions use different grids)
        fname_weight_tmp    = dir_tmp + 'weights_' + model + '_regSLAND_tmp.nc'
        fname_weight_regrid = dir_tmp + 'weights_' + model + '_regridded_regSLAND_tmp.nc'
        if os.path.exists(fname_weight_tmp):     os.remove(fname_weight_tmp)
        if os.path.exists(fname_weight_regrid):  os.remove(fname_weight_regrid)

        #Create temporary reference grid and set grid of weighting factor
        fname_grid_tmp = dir_tmp + 'grid_xy_regSLAND_tmp_' + model 
        if os.path.exists(fname_grid_tmp): os.remove(fname_grid_tmp)
        os.system('cdo -s griddes -selvar,SLAND ' + fname + ' > ' + fname_grid_tmp)
        os.system('cdo -s setgrid,' + fname_grid_tmp + ' ' + fname_weight + ' ' + fname_weight_tmp)

        #Remap weighting factors to DGVM grid 
        os.system('cdo -s remapcon,' + file_grid + ' ' + fname_weight_tmp + ' ' + fname_weight_regrid)

        #Read regridded weighting factors
        for_weight = xr.open_dataset(fname_weight_regrid)
        weight     = for_weight.w_FF

    else:

        #Flag for deleting files
        del_files = 2

        #Read weighting factors of all other DGVMs and calculate multi-model mean
        fname_weight = dir_forest + 'Weights_ForestFraction_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
        for_weight = xr.open_dataset(fname_weight)
        for_weight = for_weight.mean('model')

        #Define file names for remapping
        fname_weight_tmp1   = dir_tmp + 'weights_' + model + '_regSLAND_tmp1.nc'
        fname_weight_tmp2   = dir_tmp + 'weights_' + model + '_regSLAND_tmp2.nc'
        fname_weight_regrid = dir_tmp + 'weights_' + model + '_regridded_regSLAND_tmp.nc'
        if os.path.exists(fname_weight_tmp1):    os.remove(fname_weight_tmp1)
        if os.path.exists(fname_weight_tmp2):    os.remove(fname_weight_tmp2)
        if os.path.exists(fname_weight_regrid):  os.remove(fname_weight_regrid)

        #Save multi-model mean as NetCDF and set correct grid
        for_weight.to_netcdf(fname_weight_tmp1)
        file_grid_FM = dir_grids  + 'grid_xy_360x720'
        os.system('cdo -s setgrid,' + file_grid_FM + ' ' + fname_weight_tmp1 + ' ' + fname_weight_tmp2)

        #Remap weighting factors to model grid
        os.system('cdo -s remapcon,' + file_grid + ' ' + fname_weight_tmp2 + ' ' + fname_weight_regrid)

        #Read regridded weighting factors
        for_weight = xr.open_dataset(fname_weight_regrid)
        weight     = for_weight.w_FF

    #Define coorcinate names
    if 'latitude' in data_SLAND.coords:  lat_name, lon_name = 'latitude', 'longitude'
    else:                          lat_name, lon_name = 'lat', 'lon'
        
    #Re-index data if necessary
    check_lat1 = np.max(np.abs(weight[lat_name].values - data_SLAND[lat_name].values))
    check_lon1 = np.max(np.abs(weight[lon_name].values - data_SLAND[lon_name].values))
    check_lat2 = np.max(np.abs(data_area[lat_name].values - data_SLAND[lat_name].values))
    check_lon2 = np.max(np.abs(data_area[lon_name].values - data_SLAND[lon_name].values))    
    if check_lat1>0.001 or check_lon1>0.001:  sys.exit('Coordinates do not agree')
    if check_lat2>0.001 or check_lon2>0.001:  sys.exit('Coordinates do not agree')
    weight    = weight.reindex({lat_name: data_SLAND[lat_name], lon_name: data_SLAND[lon_name]}, method='nearest')
    data_area = data_area.reindex({lat_name: data_SLAND[lat_name], lon_name: data_SLAND[lon_name]}, method='nearest')

    #Define weight for all land and apply weighting
    weight = weight.where(weight>0, 1)
    
    #Calculate weighted SLAND density
    data_SLAND = data_SLAND * weight / data_area.cell_area   
    
    #Define file names for regridding
    fname_tmp      = dir_tmp + 'SLAND_' + model + '_tmp.nc'
    fname_tmp_regr = dir_tmp + 'SLAND_' + model + '_tmp_regr.nc'
    if os.path.exists(fname_tmp):       os.remove(fname_tmp)
    if os.path.exists(fname_tmp_regr):  os.remove(fname_tmp_regr)

    #Save SLAND density
    data_SLAND.to_netcdf(fname_tmp)
    
    #Remap SLAND to common grid
    os.system('cdo -s remapcon,' + file_grid_common + ' ' + fname_tmp + ' ' + fname_tmp_regr)    

    #Read regridded data
    data_SLAND_regr = xr.open_dataset(fname_tmp_regr).load()
    
    #Collect in array
    if create==1:
        data_coll = data_SLAND_regr
        create = 0
    else:
        data_coll = xr.concat((data_coll, data_SLAND_regr), dim='model')

    #Remove temporary files
    os.remove(fname_tmp)
    os.remove(fname_tmp_regr)
    if del_files==1:
        os.remove(fname_grid_tmp)
        os.remove(fname_weight_tmp)
        os.remove(fname_weight_regrid)
    elif del_files==2:        
        os.remove(fname_weight_tmp1)
        os.remove(fname_weight_tmp2)
        os.remove(fname_weight_regrid)
        
#Add model names
data_coll['model'] = models

#Calculate flux in Pg C / ha / year ([data_coll] =  Pg C / m2 / year)
data_coll_dens = 10**4 * data_coll

#Convert Pg C / ha / year to t C / ha / year
data_coll_dens = 10**9 * data_coll_dens

#Set attributes
data_coll_dens['SLAND'].attrs['standard_name'] = 'Natural land C sink'
data_coll_dens['SLAND'].attrs['long_name']     = 'natural_land_C_sink'
data_coll_dens['SLAND'].attrs['units']         = 't C / ha / year'

#Save as NetCDF
fname_out = dir_SLAND + "SLAND-weighted_ForestFraction_DGVMs-S2-S3_all-models_" + time_sta + "-" + time_end + "-mean_regrid" + gridsize + ".nc"
if os.path.exists(fname_out):  os.remove(fname_out)
data_coll_dens.to_netcdf(fname_out, encoding={var: comp for var in data_coll_dens.data_vars})
